In [ ]:
import os

import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          BitsAndBytesConfig, HfArgumentParser,
                          TrainingArguments, logging, pipeline)
from trl import SFTTrainer

In [ ]:
base_model_name = "meta-llama/Llama-2-7b-chat-hf"
new_model_name = "llama-2-7b-chat-hf-phr_mental_therapy-2"

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cpu", ## device map = "cpu", for merging on cpu
)
tokenizer = AutoTokenizer.from_pretrained(new_model_name)
base_model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(base_model, new_model_name)
model = model.merge_and_unload()

In [ ]:
model.push_to_hub(new_model_name, use_temp_dir=True)
tokenizer.push_to_hub(new_model_name, use_temp_dir=True)